# Segmentation

## Importing dependencies

In [ ]:
import numpy as np
from keras.models import load_model
from skimage import morphology, io, color, exposure, img_as_float, transform
from matplotlib import pyplot as plt
import openslide as op
from PIL import Image
from util import IoU, Dice, masked, remove_small_regions, normalize
from scipy import misc

import time
import h5py

## Configuration

In [ ]:
# Paths
filename = '/root/workspace/data/preExtraction/12_02/IFTA_12_02.svs'
maskname = '/root/workspace/data/preExtraction/12_02/IFTA_12_02.png'
h5f = h5py.File('/root/workspace/data/old/Experiments/Extraction/best_mask_10-11-12_index2_64/matrice_train.h5','r')
output_name="12_64_i2_s16_f1.png"

folder_output = "../data/trained/TRUE_UNET/"
model_name = folder_output+ "matrices/" + 'model.099.hdf5'

# Do not touch
level=2
patchSize = 64
stride = patchSize//4

colorChannels = 3

patchShape = (patchSize, patchSize,colorChannels) # Shape of the image (patch)
patchMaskShape = (patchSize, patchSize)   # Shape of the mask (patch)

## Image loading

### Loading base picture

In [ ]:
im = op.OpenSlide(filename)
imload = np.asfarray(im.read_region((0,0), level, im.level_dimensions[level]),dtype=np.float32)[:,:,0:colorChannels]/255

In [ ]:
print(im.level_dimensions)

In [ ]:
print(imload.mean())
print(imload.std())

In [ ]:
#normalization
stats = h5f['stats'][:]
im_mean=stats[0]
im_std=stats[1]
h5f.close()

print(im_mean)
print(im_std)
print(imload.mean())
print(imload.std())

imload-=imload.mean()
imload/=imload.std()
#imload-=im_mean
#imload/=im_std

In [ ]:
print(imload.mean())
print(imload.std())

### Loading mask

In [ ]:
maskload = Image.open(maskname)
maskload = np.expand_dims(np.array(maskload.resize(im.level_dimensions[level]),dtype=np.float32),-1)

### Allocating final array

In [ ]:
outputProba = np.zeros((im.level_dimensions[level][1],im.level_dimensions[level][0]),dtype=np.float32)
outputProbaTimes = np.zeros((im.level_dimensions[level][1],im.level_dimensions[level][0]),dtype=np.uint8)

## Loading model

In [ ]:
# Load model
UNet = load_model(model_name)

## Execution

### Prediction

In [ ]:
t0 = time.clock()


# Prediction
for posX in range(0,im.level_dimensions[level][0]-patchSize,stride):
    for posY in range(0,im.level_dimensions[level][1]-patchSize,stride):

        xx = np.expand_dims(imload[posY:posY+patchSize,posX:posX+patchSize,:],0)
        yy = maskload[posY:posY+patchSize,posX:posX+patchSize,:]
        #xx = xx[:,:,:,0:1]
        pred = UNet.predict(xx)[..., 0].reshape(patchShape[:2])

        part=outputProba[posY:posY+patchSize,posX:posX+patchSize]
        part[:,:]+=pred[0:part.shape[0],0:part.shape[1]]
        outputProbaTimes[posY:posY+patchSize,posX:posX+patchSize]+=1

t = time.clock()
print("Execution duration : ", t-t0)

### code that follows in progress 

In [ ]:
outputProba.shape

In [ ]:
outputProba/=outputProbaTimes

In [ ]:
np.putmask(outputProba, outputProba != outputProba, 0)


In [ ]:
print(outputProba.min())
print(outputProba.max())

In [ ]:
outputProbaThresh = outputProba > 0.8
maskTresh = maskload > 0.5

In [ ]:
outputProbaThresh = remove_small_regions(outputProbaThresh, 0.02 * np.prod(patchShape))

In [ ]:
testDelAmore = np.dstack((outputProbaThresh.astype(np.int8), maskTresh.astype(np.int8), outputProbaThresh.astype(np.int8)))

In [ ]:
plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
plt.subplot(211)
plt.imshow(outputProba,interpolation='none', cmap="jet")
plt.show()

#plt.close()

In [ ]:
plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
plt.subplot(211)
plt.imshow(testDelAmore,interpolation='none', cmap="jet")
plt.show()

In [ ]:
misc.imsave(folder_output + "prediction/" + output_name, testDelAmore)

In [ ]:
#Trying to save the file on the fly
#maskImage = Image.fromarray(testDelAmore, 'RGB')
#maskImage.save("../data/prediction/prediction_14-02_256_0.9_10-11-13-200-100.png")